In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the CSV file
file_path = input("Please enter the input each_trait_CSV by calculate_pearsonrs_spearmanrs_p-values.jpynb path: ")
df = pd.read_csv(file_path)

# Extract the unique stimulus sentences
stimuli = pd.unique(df['sorted_pair'].apply(lambda x: eval(x)))

# Create an empty matrix with rows and columns representing the stimuli
stimuli_list = sorted(set([stimulus for pair in stimuli for stimulus in pair]))
matrix = pd.DataFrame(index=stimuli_list, columns=stimuli_list)

# Create a matrix for the p-values as well
p_value_matrix = pd.DataFrame(index=stimuli_list, columns=stimuli_list)

# Fill in the matrix with Spearman correlation coefficients and p-values
for i, row in df.iterrows():
    stimulus1, stimulus2 = eval(row['sorted_pair'])
    spearman_corr = row['spearman_corr']
    spearman_p_value = row['spearman_p_value']
    
    # Fill the matrix symmetrically for both (stimulus1, stimulus2) and (stimulus2, stimulus1)
    matrix.loc[stimulus1, stimulus2] = spearman_corr
    matrix.loc[stimulus2, stimulus1] = spearman_corr
    
    # Fill the p-value matrix
    p_value_matrix.loc[stimulus1, stimulus2] = spearman_p_value
    p_value_matrix.loc[stimulus2, stimulus1] = spearman_p_value

# Convert matrices to numeric, replacing NaNs
matrix = matrix.apply(pd.to_numeric, errors='coerce')
p_value_matrix = p_value_matrix.apply(pd.to_numeric, errors='coerce')

# Function to apply color based on the correlation coefficient and p-value
def color_cells(val, pval):
    if pval < 0.05:  # Only apply color if p-value is significant
        if 0.7 <= val <= 1:
            return 'background-color: red'
        elif 0.4 <= val < 0.7:
            return 'background-color: lightcoral'
        elif 0.2 <= val < 0.4:
            return 'background-color: #fad598'
        elif -0.2 <= val < 0.2:
            return 'background-color: #D3D3D3'
        elif -0.4 <= val < -0.2:
            return 'background-color: #90EE90'
        elif -0.7 <= val < -0.4:
            return 'background-color: lightblue'
        elif -1 <= val < -0.7:
            return 'background-color: blue'
    return ''  # No color if the p-value is not significant or outside defined ranges

# Applying the function to the matrix, using p-value as a condition
styled_matrix = matrix.style.apply(lambda x: np.vectorize(color_cells)(x, p_value_matrix), axis=None)

# Optionally, save the matrix with color to an Excel file for better visualization
output_file_path = input("Please enter the output excel file path: ")
styled_matrix.to_excel(output_file_path, engine='openpyxl')

# Provide the path to the saved Excel file
output_file_path